In [1]:
import pandas as pd
import numpy as np
from fastDamerauLevenshtein import damerauLevenshtein
import ast
import math

In [2]:
class Prediction():
    Item = None
    Parent = None
    Children = None

    def __init__(self, itemValue=None):
        self.Item = itemValue
        self.Children = []
        self.Parent = None

    def get_child(self, target):
        for chld in self.Children:
            if chld.Item == target:
                return chld
        return None

    def get_children(self):
        return self.Children

    def has_child(self, target):
        found = self.get_child(target)
        if found is not None:
            return True
        else:
            return False

    def add_new_child(self, child):
        newchild = Prediction(child)
        newchild.Parent = self
        self.Children.append(newchild)

    def remove_child(self, child):
        for chld in self.Children:
            if chld.Item == child:
                self.Children.remove(chld)

In [134]:
class CPT():
    alphabet = None
    root = None
    inverted_index = None
    lookup_table = None
    def __init__(self):
        self.alphabet = set()
        self.root = Prediction()
        self.inverted_index = {}
        self.lookup_table = {}

    def load_files(self, train_file, test_file=None):
        """
        This function reads in the wide csv file of sequences separated by commas and returns a list of list of 
        those sequences. The sequences are defined as below.
        seq1 = A,B,C,D
        seq2 = B,C,E
        Returns: [[A,B,C,D],[B,C,E]]
        """
        
        train = []
        test = []

        if train_file is None:
            return train_file

        train_data = pd.read_csv(train_file, header=0)

        for index, row in train_data.iterrows():
            train.append(row.values)

        if test_file is not None:

            test_data = pd.read_csv(test_file, header=None)

            for index, row in test_data.iterrows():
                train.append(row.values)
                test.append(list(row.values))

            return train, test

        return train


    def train(self, data):
        """
         This functions populates the Prediction Tree, Inverted Index and LookUp Table for the algorithm.
         Input: The list of list training data
         Output : Boolean True
         """

        cursornode = self.root

        for seqid, row in enumerate(data):
            for element in row:
                if element == element: # different sequence length support
                    
                    if cursornode.has_child(element) == False:
                        cursornode.add_new_child(element)
                        cursornode = cursornode.get_child(element)
                    else:
                        cursornode = cursornode.get_child(element)

                # adding to the inverted index

                    if self.inverted_index.get(element) is None:
                        self.inverted_index[element] = set()

                    self.inverted_index[element].add(seqid)

                    self.alphabet.add(element)

            self.lookup_table[seqid] = cursornode

            cursornode = self.root

        return True

    def score(self, counttable, key, length, target_size, number_of_similar_sequences, number_items_counttable):
        """
         This functions populates the Prediction Tree, Inverted Index and LookUp Table for the algorithm.
         Input: The list of list training data
         Output : count table
         """
        
        weight_level = 1 / number_of_similar_sequences
        weight_distance = 1 / number_items_counttable
        score = 1 + weight_level + weight_distance * 0.001

        if counttable.get(key) is None:
            counttable[key] = score
        else:
            counttable[key] = score * counttable.get(key)

        return counttable

    def predict(self, data, target, k, n):
        """
        Here target is the test dataset in the form of list of list,
        k is the number of last elements that will be used to find similar sequences and,
        n is the number of predictions required.
        Input: training list of list, target list of list, k,n
        Output: max n predictions for each sequence
        """

        predictions = []

        for index, each_target in enumerate(target):
            #n = int(n_list[index])
            
            # different sequence length support
            i = 0
            while i < len(each_target) and each_target[i] == each_target[i]:  # find NaN start
                i = i + 1
            l = i - k - 1
            if l < 0:
                l = 0
            
            each_target = each_target[l:i]

            intersection = set(range(0, len(data)))

            for element in each_target:
                if self.inverted_index.get(element) is None:
                    continue
                intersection = intersection & self.inverted_index.get(element)

            similar_sequences = []

            for element in intersection:
                currentnode = self.lookup_table.get(element)
                tmp = []
                while currentnode.Item is not None:
                    tmp.append(currentnode.Item)
                    currentnode = currentnode.Parent
                similar_sequences.append(tmp)

            for sequence in similar_sequences:
                sequence.reverse()

            counttable = {}

            for sequence in similar_sequences:
                try:
                    index = next(
                        i for i, v in zip(range(len(sequence) - 1, 0, -1), reversed(sequence)) if v == each_target[-1])
                except:
                    index = None
                if index is not None:
                    count = 1
                    for element in sequence[index + 1:]:
                        if element in each_target:
                            continue

                        counttable = self.score(counttable, element, len(each_target), len(each_target),
                                                len(similar_sequences), count)
                        #print(counttable)
                        count += 1

            pred = self.get_n_largest(counttable, n)
            predictions.append(pred)

        return predictions

    def get_n_largest(self, dictionary, n):

        largest = sorted(dictionary.items(), key=lambda t: t[1], reverse=True)[:n]
        return [key for key, _ in largest]

In [135]:
model = CPT()

In [156]:
train, test = model.load_files('data/all_data_train_firstelem.csv', 'data/all_data_test_7_firstelem.csv')

In [157]:
model.train(train)

True

In [158]:
# get length of each sequence (ignoring nan) minus given sequence in test
n_list = [np.count_nonzero(~pd.isnull(x)) - 1 for x in train]
n_list = [x if x > 0 else 0 for x in n_list]

In [60]:
def get_string(string):
    output = ''
    for lst in string:
        for elem in lst:
            output += elem
    return output

### Binary evaluation of errors in each step

In [159]:
errors = [[] for x in range(0,len(train)//2)]

for seq in range(0,len(train)//2):
    i = 1
    while i <= n_list[seq]:
    #while i <= 2:
        next_char = model.predict(train, train[seq][i-1:i], 3, 1)
        print('input:', train[seq][i-1:i])
        #print('next char: ' , next_char)
        next_char = get_string(next_char)
        predicted = next_char
        observed = train[seq][i]
        print('pred: ', predicted, 'obs: ', observed, i)
        #print('\n -----------------')
        #print(counttable)
        #print('\n -----------------')
        error = 1 - damerauLevenshtein(predicted, observed)
        #print(error, '\n' , '____________')
        
        errors[seq].append(error)
        i += 1
    print('\n___________________')

input: ['j']
pred:   obs:  p 1
input: ['p']
pred:  k obs:  o 2
input: ['o']
pred:  k obs:  c 3
input: ['c']
pred:  k obs:  g 4
input: ['g']
pred:  k obs:  k 5
input: ['k']
pred:  s obs:  r 6

___________________
input: ['j']
pred:   obs:  c 1
input: ['c']
pred:  k obs:  g 2
input: ['g']
pred:  k obs:  w 3
input: ['w']
pred:  k obs:  p 4
input: ['p']
pred:  k obs:  c 5
input: ['c']
pred:  k obs:  f 6
input: ['f']
pred:  s obs:  k 7
input: ['k']
pred:  s obs:  s 8

___________________
input: ['j']
pred:   obs:  k 1
input: ['k']
pred:  s obs:  f 2
input: ['f']
pred:  s obs:  s 3
input: ['s']
pred:  g obs:  f 4
input: ['f']
pred:  s obs:  k 5
input: ['k']
pred:  s obs:  s 6
input: ['s']
pred:  g obs:  p 7
input: ['p']
pred:  k obs:  w 8
input: ['w']
pred:  k obs:  g 9

___________________
input: ['j']
pred:   obs:  p 1
input: ['p']
pred:  k obs:  f 2
input: ['f']
pred:  s obs:  k 3
input: ['k']
pred:  s obs:  s 4
input: ['s']
pred:  g obs:  w 5
input: ['w']
pred:  k obs:  k 6
input: ['k']


pred:  k obs:  o 2
input: ['o']
pred:  k obs:  w 3
input: ['w']
pred:  k obs:  g 4
input: ['g']
pred:  k obs:  c 5
input: ['c']
pred:  k obs:  p 6
input: ['p']
pred:  k obs:  o 7

___________________
input: ['q']
pred:   obs:  p 1
input: ['p']
pred:  k obs:  w 2
input: ['w']
pred:  k obs:  f 3
input: ['f']
pred:  s obs:  k 4
input: ['k']
pred:  s obs:  t 5

___________________
input: ['q']
pred:   obs:  p 1
input: ['p']
pred:  k obs:  w 2
input: ['w']
pred:  k obs:  o 3
input: ['o']
pred:  k obs:  k 4
input: ['k']
pred:  s obs:  f 5
input: ['f']
pred:  s obs:  k 6
input: ['k']
pred:  s obs:  r 7
input: ['r']
pred:  k obs:  g 8

___________________
input: ['q']
pred:   obs:  c 1
input: ['c']
pred:  k obs:  o 2
input: ['o']
pred:  k obs:  p 3
input: ['p']
pred:  k obs:  c 4
input: ['c']
pred:  k obs:  o 5
input: ['o']
pred:  k obs:  g 6
input: ['g']
pred:  k obs:  p 7
input: ['p']
pred:  k obs:  g 8
input: ['g']
pred:  k obs:  w 9
input: ['w']
pred:  k obs:  f 10
input: ['f']
pred:  s ob

pred:   obs:  t 1
input: ['t']
pred:  f obs:  g 2
input: ['g']
pred:  k obs:  c 3
input: ['c']
pred:  k obs:  w 4
input: ['w']
pred:  k obs:  p 5
input: ['p']
pred:  k obs:  r 6
input: ['r']
pred:  k obs:  o 7

___________________
input: ['q']
pred:   obs:  p 1
input: ['p']
pred:  k obs:  o 2
input: ['o']
pred:  k obs:  c 3
input: ['c']
pred:  k obs:  g 4
input: ['g']
pred:  k obs:  k 5
input: ['k']
pred:  s obs:  f 6
input: ['f']
pred:  s obs:  s 7
input: ['s']
pred:  g obs:  k 8
input: ['k']
pred:  s obs:  f 9
input: ['f']
pred:  s obs:  r 10

___________________
input: ['q']
pred:   obs:  p 1
input: ['p']
pred:  k obs:  g 2
input: ['g']
pred:  k obs:  c 3
input: ['c']
pred:  k obs:  o 4
input: ['o']
pred:  k obs:  w 5

___________________
input: ['q']
pred:   obs:  o 1
input: ['o']
pred:  k obs:  p 2
input: ['p']
pred:  k obs:  w 3
input: ['w']
pred:  k obs:  g 4

___________________
input: ['q']
pred:   obs:  w 1
input: ['w']
pred:  k obs:  p 2
input: ['p']
pred:  k obs:  g 3
input

input: ['k']
pred:  s obs:  s 4
input: ['s']
pred:  g obs:  c 5
input: ['c']
pred:  k obs:  a 6
input: ['a']
pred:  b obs:  g 7
input: ['g']
pred:  k obs:  b 8

___________________
input: ['q']
pred:   obs:  p 1
input: ['p']
pred:  k obs:  f 2
input: ['f']
pred:  s obs:  s 3
input: ['s']
pred:  g obs:  k 4
input: ['k']
pred:  s obs:  a 5
input: ['a']
pred:  b obs:  g 6
input: ['g']
pred:  k obs:  c 7
input: ['c']
pred:  k obs:  b 8

___________________
input: ['q']
pred:   obs:  p 1
input: ['p']
pred:  k obs:  s 2
input: ['s']
pred:  g obs:  k 3
input: ['k']
pred:  s obs:  f 4
input: ['f']
pred:  s obs:  c 5
input: ['c']
pred:  k obs:  a 6
input: ['a']
pred:  b obs:  g 7
input: ['g']
pred:  k obs:  b 8

___________________
input: ['q']
pred:   obs:  p 1
input: ['p']
pred:  k obs:  f 2
input: ['f']
pred:  s obs:  k 3
input: ['k']
pred:  s obs:  s 4
input: ['s']
pred:  g obs:  c 5
input: ['c']
pred:  k obs:  a 6
input: ['a']
pred:  b obs:  g 7
input: ['g']
pred:  k obs:  b 8

___________

input: ['g']
pred:  k obs:  c 8

___________________
input: ['q']
pred:   obs:  p 1
input: ['p']
pred:  k obs:  k 2
input: ['k']
pred:  s obs:  f 3
input: ['f']
pred:  s obs:  s 4
input: ['s']
pred:  g obs:  g 5
input: ['g']
pred:  k obs:  c 6
input: ['c']
pred:  k obs:  b 7
input: ['b']
pred:  a obs:  a 8

___________________
input: ['q']
pred:   obs:  p 1
input: ['p']
pred:  k obs:  k 2
input: ['k']
pred:  s obs:  f 3
input: ['f']
pred:  s obs:  s 4
input: ['s']
pred:  g obs:  g 5
input: ['g']
pred:  k obs:  c 6
input: ['c']
pred:  k obs:  b 7
input: ['b']
pred:  a obs:  a 8

___________________
input: ['q']
pred:   obs:  p 1
input: ['p']
pred:  k obs:  k 2
input: ['k']
pred:  s obs:  f 3
input: ['f']
pred:  s obs:  s 4
input: ['s']
pred:  g obs:  g 5
input: ['g']
pred:  k obs:  c 6
input: ['c']
pred:  k obs:  a 7
input: ['a']
pred:  b obs:  b 8

___________________
input: ['q']
pred:   obs:  p 1
input: ['p']
pred:  k obs:  k 2
input: ['k']
pred:  s obs:  f 3
input: ['f']
pred:  s ob

In [160]:
summed_error = [sum(error) for error in errors]
np.median(summed_error)

7.0

In [67]:
with open('results/cpt_prequential_summed_startfromzero.txt', 'w') as f:
    f.write(str(summed_error))

In [144]:
len(errors)

180